In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
 import os
 os.chdir("/content/drive/MyDrive/Colab Notebooks/DBSE Project")

In [4]:
pip install python-dotenv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
pip install git+https://github.com/tweepy/tweepy.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/tweepy/tweepy.git to /tmp/pip-req-build-da_pepbb
  Running command git clone -q https://github.com/tweepy/tweepy.git /tmp/pip-req-build-da_pepbb
     |████████████████████████████████| 62 kB 912 kB/s 
  Created wheel for tweepy: filename=tweepy-4.12.0-py3-none-any.whl size=101665 sha256=f96812434cbcb4c4b64e6bcae00e446b694ea2d4cf6c768327ed0a1736df2aea
  Stored in directory: /tmp/pip-ephem-wheel-cache-v2dqi1ty/wheels/b4/a5/5a/5074abdb9f4bd5bd0e22631a63fc41ae2fa71ad83780ea18d1
Successfully built tweepy
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: tweepy
    Found existing installation: tweepy 3.10.0
    Uninstalling tweepy-3.10.0:
      Successfully uninstalled tweepy-3.10.0


In [14]:
pip install snscrape

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
import pandas as pd
import numpy as np
import tweepy
import os
from dotenv import load_dotenv
import csv
import time
import snscrape.modules.twitter as sntwitter

In [7]:
#Fetching required Keys
load_dotenv()

consumer_key = os.environ["API_KEY"]
consumer_secret = os.environ["API_KEY_SECRET"]
access_token = os.environ["ACCESS_TOKEN"]
access_token_secret = os.environ["ACCESS_TOKEN_SECRET"]
bearer_token = os.environ['BEARER_TOKEN']

In [8]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

# You can authenticate as your app with just your bearer token
client = tweepy.Client(bearer_token=bearer_token)
client = tweepy.Client(
    consumer_key=consumer_key, consumer_secret=consumer_secret,
    access_token=access_token, access_token_secret=access_token_secret, 
    bearer_token=bearer_token
)

In [53]:
query = "(from:#IchBinHanna) until:2022-10-30 since:2021-07-01"
tweets = []
limit = 1000
for tweet in sntwitter.TwitterSearchScraper(query).get_items():
    if len(tweets) == limit:
        break
    else:
        tweets.append([tweet.id, tweet.date, tweet.username, tweet.content,
                       tweet.url])

tweet_data=[]
result=[]
for data in tweets:
  get_tweets = client.get_tweets(ids=data[0], tweet_fields=['context_annotations','created_at','geo'], expansions='author_id')
  tweet_data=[data[0], data[1], data[2], data[3],data[4]]
  for tweet1 in get_tweets.includes['users']:
    users = client.get_users(ids=tweet1.id, user_fields=['username','public_metrics','description', 'location', 'profile_image_url'])
    for user in users.data:
      result.append({'Tweet_id': tweet_data[0],
                     'Tweet_date':tweet_data[1],
                     'Author_id':tweet1.id,
                     'Username': user.username,
                     'Author_description': user.description,
                     'Author_location': user.location,
                     'Author_followers': user.public_metrics['followers_count'],
                     'Author_following_count': user.public_metrics['following_count'],
                     'Author_tweet_count': user.public_metrics['tweet_count'],
                     'Tweet_content': tweet_data[3],
                     'Tweet_URL': tweet_data[4],
                     'Author_image_URL': user.profile_image_url
                      })
df = pd.DataFrame(result)
df
df.to_csv('/content/drive/MyDrive/Colab Notebooks/DBSE Project/twitterdata.csv')